In [1]:
# import sys
# !{sys.executable} -m pip install robobrowser
import requests
from requests.auth import HTTPBasicAuth
import json
import datetime
from dateutil import parser

In [11]:
req = requests.get('http://expertpowerplus.com:8080/api/Login?userName=ppl&pass=Wyre1234')

In [21]:
auth_key_name = (list(req.cookies)[0]).name #get name of cookie unit used to be (.ASPXAUTH) chnaged to (form_p)
auth_key_value = dict(req.cookies).get(auth_key_name) #get actual cookie unit

In [31]:
cookie = {auth_key_name: auth_key_value}
device = "125639"#"133813" #128166
url_lastreading = f'http://expertpowerplus.com:8080/api/basic/{device}/LastReading'

url_period_reading = f'http://expertpowerplus.com:8080/api/Basic?startDate=2019-08-11&endDate=c&deviceId={device}'
url_data_logs = f'http://www.expertpowerplus.com:8080/api/basic/{device}/Datalogs?startDate=2019-07-01&endDate=2019-07-31&datalogNum=1'

r = requests.get(url_data_logs, cookies=cookie)
print(r)

<Response [200]>


In [32]:
data = r.json()
data

{'resultCode': 200,
 'message': None,
 'data': [{'deviceID': 125639,
   'recordTime': '2019-07-30T23:45:00',
   'data': [{'description': 'Digital Input #1',
     'value': 0.0,
     'units': '&nbsp;'},
    {'description': 'Summary Energy Register #1',
     'value': 22723.0,
     'units': ''},
    {'description': 'Summary Energy Register #2', 'value': 305.0, 'units': ''},
    {'description': 'Total kW', 'value': 8.0, 'units': 'kW'}]},
  {'deviceID': 125639,
   'recordTime': '2019-07-30T23:30:00',
   'data': [{'description': 'Digital Input #1',
     'value': 1.0,
     'units': '&nbsp;'},
    {'description': 'Summary Energy Register #1',
     'value': 22720.0,
     'units': ''},
    {'description': 'Summary Energy Register #2', 'value': 298.0, 'units': ''},
    {'description': 'Total kW', 'value': 12.0, 'units': 'kW'}]},
  {'deviceID': 125639,
   'recordTime': '2019-07-30T23:15:00',
   'data': [{'description': 'Digital Input #1',
     'value': 1.0,
     'units': '&nbsp;'},
    {'descriptio

In [33]:
readings = data['data'][0]['data']
utility_times = []
gen_times = []

for readings in reversed(data['data']):
    record_time = readings["recordTime"]
    digital_in = readings["data"][0]["value"]
    energy_1 = readings["data"][1]["value"]
    energy_2 = readings["data"][2]["value"]
    total_energy = readings["data"][3]["value"]
    
    if digital_in == 0:
        utility_times.append((record_time, energy_1))
        
    elif digital_in == 1:
        gen_times.append((record_time, energy_1))

In [34]:
def get_time_dif(a, b):
    
    c = b - a
    hours = c.seconds/3600
    return hours

In [35]:
def format_date(date):###THIS FUNCTION CONVERTS DATE FROM DD-MM-YYY TO YYY-MM-DD

    datetime_object = parser.parse(date)

    return datetime_object

In [36]:
target_data = data["data"]
df_data = {}

def rearrange_data(data):
    for item in data:
        day = format_date(item["recordTime"])
        if df_data.get(day.day, False):
#             print(day.day)
            df_data[day.day].append(item)
        else:
            df_data[day.day] = []
            df_data[day.day].append(item)
        
rearrange_data(target_data)
# print(df_data)

In [37]:
nums = data["data"].copy()
nums.reverse()
utility_kwh = []
gen1_kwh = []
gen2_kwh = []

# print(nums)
while len(nums) > 1:
    x,y = nums.pop(0), nums[0]
    
    x_di = x["data"][0]["value"]
    y_di = y["data"][0]["value"]
    x_time, y_time = format_date(x["recordTime"]), format_date(y["recordTime"])
    x_kwh, y_kwh = x["data"][1]["value"], y["data"][1]["value"]
    
    if x_di == 0 and y_di == 1 or y_di == 2:
        time_diff = get_time_dif(x_time, y_time)
        if y_di == 1:
            gen1_hrs.append(time_diff)
        
        elif y_di == 2:
            gen2_hrs.append(time_diff)

        
    elif x_di == 1 and y_di == 0:
        time_diff = get_time_dif(x_time, y_time)
        utility_hrs.append(time_diff)
        
        
    elif x_di == y_di == 1 or x_di == y_di == 2:
        time_diff = get_time_dif(x_time, y_time)
        if y_di == 1:
            gen1_hrs.append(time_diff)
        
        elif y_di == 2:
            gen2_hrs.append(time_diff)

    elif x_di == y_di == 0:
        time_diff = get_time_dif(x_time, y_time)
        utility_hrs.append(time_diff)
        
    
print(sum(utility_hrs), sum(gen1_hrs), sum(gen2_hrs))
print(sum(utility_kwh), sum(gen1_kwh), sum(gen2_kwh))
# print("\n")

NameError: name 'utility_hrs' is not defined

In [ ]:
nums = data["data"].copy()
# nums.reverse()

def get_daily_usage(data):
    data.reverse()
    utility_hrs = []
    utility_kwh = []
    gen1_hrs = []
    gen1_kwh = []
    gen2_hrs = []
    gen2_kwh = []
    while len(data) > 1:
        
        x,y = data.pop(0), data[0]

        x_di = x["data"][0]["value"]
        y_di = y["data"][0]["value"]
        x_time, y_time = format_date(x["recordTime"]), format_date(y["recordTime"])
        x_kwh, y_kwh = x["data"][1]["value"], y["data"][1]["value"]

        if x_di == 0 and y_di == 1 or y_di == 2:
            time_diff = get_time_dif(x_time, y_time)
            kwh_diff = y_kwh - x_kwh
            if y_di == 1:
                gen1_hrs.append(time_diff)
                gen1_kwh.append(kwh_diff)

            elif y_di == 2:
                gen2_hrs.append(time_diff)
                gen2_kwh.append(kwh_diff)


        elif x_di == 1 and y_di == 0:
            time_diff = get_time_dif(x_time, y_time)
            kwh_diff = y_kwh -x_kwh
            utility_hrs.append(time_diff)
            utility_kwh.append(kwh_diff)


        elif x_di == y_di == 1 or x_di == y_di == 2:
            time_diff = get_time_dif(x_time, y_time)
            kwh_diff = y_kwh - x_kwh
            if y_di == 1:
                gen1_hrs.append(time_diff)
                gen1_kwh.append(kwh_diff)

            elif y_di == 2:
                gen2_hrs.append(time_diff)
                gen2_kwh.append(kwh_diff)

        elif x_di == y_di == 0:
            time_diff = get_time_dif(x_time, y_time)
            kwh_diff = y_kwh - x_kwh
            utility_hrs.append(time_diff)
            utility_kwh.append(kwh_diff)
    print (sum(utility_kwh), sum(gen1_kwh), sum(gen2_kwh))
    print (sum(utility_hrs), sum(gen1_hrs), sum(gen2_hrs))
    return (sum(utility_kwh), sum(gen1_kwh), sum(gen2_kwh))
# get_daily_usage(nums)
# print(sum(utility_hrs), sum(gen1_hrs), sum(gen2_hrs))
# print(sum(utility_kwh), sum(gen1_kwh), sum(gen2_kwh))
# print("\n")

In [38]:
daily_data = {}
for day in df_data:
    print("-----",day, get_daily_usage(df_data[day]))
    if daily_data.get(day,False):
        daily_data[day].append(get_daily_usage(df_data[day]))
    else:
        daily_data[day] = []
        daily_data[day].append(get_daily_usage(df_data[day]))
        
print("-----",daily_data)

371.0 337.0 0
15.75 8.0 0
----- 30 (371.0, 337.0, 0)
0 0 0
0 0 0
414.0 233.0 9.0
16.5 5.5 1.5
----- 29 (414.0, 233.0, 9.0)
0 0 0
0 0 0
91.0 0 36.0
16.5 0 7.0
----- 28 (91.0, 0, 36.0)
0 0 0
0 0 0
148.0 0 0
23.75 0 0
----- 27 (148.0, 0, 0)
0 0 0
0 0 0
476.0 109.0 0
21.5 2.25 0
----- 26 (476.0, 109.0, 0)
0 0 0
0 0 0
635.0 2.0 0
20.75 2.75 0
----- 25 (635.0, 2.0, 0)
0 0 0
0 0 0
582.0 0 28.0
19.75 0 4.0
----- 24 (582.0, 0, 28.0)
0 0 0
0 0 0
557.0 69.0 10.0
20.25 2.25 1.0
----- 23 (557.0, 69.0, 10.0)
0 0 0
0 0 0
562.0 229.0 0
19.5 4.25 0
----- 22 (562.0, 229.0, 0)
0 0 0
0 0 0
136.0 13.0 0
20.25 3.5 0
----- 21 (136.0, 13.0, 0)
0 0 0
0 0 0
138.0 0 0
23.75 0 0
----- 20 (138.0, 0, 0)
0 0 0
0 0 0
609.0 0 0
23.75 0 0
----- 19 (609.0, 0, 0)
0 0 0
0 0 0
474.0 157.0 0
19.75 4.0 0
----- 18 (474.0, 157.0, 0)
0 0 0
0 0 0
636.0 0 0
23.75 0 0
----- 17 (636.0, 0, 0)
0 0 0
0 0 0
72.0 174.0 0
7.0 5.25 0
----- 16 (72.0, 174.0, 0)
0 0 0
0 0 0
23.0 0 0
16.5 0 0
----- 15 (23.0, 0, 0)
0 0 0
0 0 0
28.0 0 0
23.75 0